In [1]:
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI
from utils import geocode_address_string


# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [3]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [10]:
import csv

all_form13s = []

with open('./data/form13.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader: # each row will be a dictionary
      all_form13s.append(row)

In [4]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
textEmbedding: LIST} Form {names: LIST, formId: STRING, cik:
STRING, cusip6: STRING, source: STRING} Company {names:
LIST, cusip6: STRING, companyName: STRING, cusip: STRING}
Manager {managerCik: STRING, managerName: STRING,
managerAddress: STRING} Relationship properties: SECTION
{f10kItem: STRING} OWNS_STOCK_IN {reportCalendarOrQuarter:
STRING, value: FLOAT, shares: INTEGER} The relationships:
(:Chunk)-[:NEXT]->(:Chunk) (:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk) (:Company)-[:FILED]->(:Form)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)


In [ ]:
kg.query("""
CREATE CONSTRAINT unique_address 
  IF NOT EXISTS
  FOR (a:Address) 
  REQUIRE a.location IS UNIQUE
""")

# Iterate over the extracted addresses and create Cypher queries


In [33]:
cypher = """
    MERGE (a:Address {location: point($address.location) })
      ON CREATE 
        SET a.city = $address.city
        SET a.state = $address.state
        SET a.country = $address.country
    WITH a
      MATCH (m:Manager)
        WHERE m.managerCik = $managerCik
    WITH a,m
      MERGE (m)-[:LOCATED_AT]->(a)
"""

In [38]:
for form13 in all_form13s:
    address = geocode_address_string(form13['managerAddress'])
    kg.query(cypher, params={'address': address, 'managerCik': form13['managerCik']})

In [45]:
kg.query("""
  MATCH (a:Address)<-[:LOCATED_AT]-(m:Manager)
  RETURN a,m LIMIT 1
""")

[{'a': {'country': 'Canada',
   'city': 'Toronto',
   'location': POINT(-79.3804133 43.6468457),
   'state': 'Ontario'},
  'm': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'}}]

In [ ]:
# add company addresses

address = {'location': {'latitude': 37.3188057, 'longitude': -121.9473604}, 'country': 'United States', 'city': 'San Jose', 'state': 'California'}

cypher = """
    MERGE (a:Address {location: point($address.location) })
      ON CREATE 
        SET a.city = $address.city
        SET a.state = $address.state
        SET a.country = $address.country
    WITH a
      MATCH (c:Company)
        WHERE c.cusip6 = $cusip6
    WITH a,c
      MERGE (c)-[:LOCATED_AT]->(a)
"""

kg.query(cypher, params={'address': address, 'cusip6': '64110D'})

[]

In [47]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
textEmbedding: LIST} Form {names: LIST, formId: STRING, cik:
STRING, cusip6: STRING, source: STRING} Company {names:
LIST, cusip6: STRING, companyName: STRING, cusip: STRING}
Manager {managerCik: STRING, managerName: STRING,
managerAddress: STRING} Address {location: POINT, city:
STRING, state: STRING, country: STRING} Relationship
properties: SECTION {f10kItem: STRING} OWNS_STOCK_IN
{reportCalendarOrQuarter: STRING, value: FLOAT, shares:
INTEGER} The relationships: (:Chunk)-[:NEXT]->(:Chunk)
(:Chunk)-[:PART_OF]->(:Form) (:Form)-[:SECTION]->(:Chunk)
(:Company)-[:FILED]->(:Form)
(:Company)-[:LOCATED_AT]->(:Address)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)
(:Manager)-[:LOCATED_AT]->(:Address)


In [48]:
kg.query("""
  MATCH (c:Company)
  RETURN c LIMIT 5
""")

[{'c': {'cusip': '64110D104',
   'names': ['Netapp Inc', 'NETAPP INC'],
   'companyName': 'NETAPP INC',
   'cusip6': '64110D'}}]

In [49]:
kg.query("""
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr, addr
LIMIT 1
""")

[{'mgr': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'},
  'addr': {'country': 'Canada',
   'city': 'Toronto',
   'location': POINT(-79.3804133 43.6468457),
   'state': 'Ontario'}}]

In [50]:
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank") YIELD node, score
  RETURN node.managerName, score LIMIT 1
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 3.7019896507263184}]

In [51]:
kg.query("""
CALL db.index.fulltext.queryNodes(
         "fullTextManagerNames", 
         "royal bank"
  ) YIELD node, score
WITH node as mgr LIMIT 1
MATCH (mgr:Manager)-[:LOCATED_AT]->(addr:Address)
RETURN mgr.managerName, addr
""")

[{'mgr.managerName': 'Royal Bank of Canada',
  'addr': {'country': 'Canada',
   'city': 'Toronto',
   'location': POINT(-79.3804133 43.6468457),
   'state': 'Ontario'}}]

In [52]:
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

[{'state': 'New York', 'numManagers': 58},
 {'state': 'California', 'numManagers': 52},
 {'state': 'Massachusetts', 'numManagers': 32},
 {'state': 'Illinois', 'numManagers': 29},
 {'state': 'Pennsylvania', 'numManagers': 29},
 {'state': 'Florida', 'numManagers': 19},
 {'state': 'Ohio', 'numManagers': 18},
 {'state': 'Texas', 'numManagers': 18},
 {'state': 'England', 'numManagers': 16},
 {'state': 'Ontario', 'numManagers': 15}]

In [53]:
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numCompanies
    ORDER BY numCompanies DESC
""")

[{'state': 'California', 'numCompanies': 1}]

In [54]:
kg.query("""
  MATCH p=(:Manager)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numManagers
    ORDER BY numManagers DESC
    LIMIT 10
""")

[{'city': 'San Francisco', 'numManagers': 10},
 {'city': 'Los Angeles', 'numManagers': 8},
 {'city': 'Pasadena', 'numManagers': 3},
 {'city': 'San Diego', 'numManagers': 3},
 {'city': 'Palos Verdes Estates', 'numManagers': 2},
 {'city': 'Oakland', 'numManagers': 2},
 {'city': 'Beverly Hills', 'numManagers': 2},
 {'city': 'San Rafael', 'numManagers': 2},
 {'city': 'Folsom', 'numManagers': 1},
 {'city': 'El Segundo', 'numManagers': 1}]

In [55]:
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numCompanies
    ORDER BY numCompanies DESC
""")

[{'city': 'San Jose', 'numCompanies': 1}]

In [56]:
kg.query("""
  MATCH p=(mgr:Manager)-[:LOCATED_AT]->(address:Address),
         (mgr)-[owns:OWNS_STOCK_IN]->(:Company)
         WHERE address.city = "San Francisco"
  RETURN mgr.managerName, sum(owns.value) as totalInvestmentValue
    ORDER BY totalInvestmentValue DESC
    LIMIT 10
""")

[{'mgr.managerName': 'CHARLES SCHWAB INVESTMENT MANAGEMENT INC',
  'totalInvestmentValue': 136175207000.0},
 {'mgr.managerName': 'WELLS FARGO & COMPANY/MN',
  'totalInvestmentValue': 15939789000.0},
 {'mgr.managerName': 'SKBA CAPITAL MANAGEMENT LLC',
  'totalInvestmentValue': 13022380000.0},
 {'mgr.managerName': 'Legato Capital Management LLC',
  'totalInvestmentValue': 2991977000.0},
 {'mgr.managerName': 'Parallax Volatility Advisers, L.P.',
  'totalInvestmentValue': 1481320000.0},
 {'mgr.managerName': 'WETHERBY ASSET MANAGEMENT INC',
  'totalInvestmentValue': 848117000.0},
 {'mgr.managerName': 'Woodline Partners LP',
  'totalInvestmentValue': 327068000.0},
 {'mgr.managerName': 'Fund Management at Engine No. 1 LLC',
  'totalInvestmentValue': 212774000.0},
 {'mgr.managerName': 'ALGERT GLOBAL LLC', 'totalInvestmentValue': 1378000.0},
 {'mgr.managerName': 'Cutler Group LLC / CA', 'totalInvestmentValue': 0.0}]

In [59]:
kg.query("""
  MATCH (com:Company)-[:LOCATED_AT]->(address:Address)
  RETURN com
""")

[{'com': {'cusip': '64110D104',
   'names': ['Netapp Inc', 'NETAPP INC'],
   'companyName': 'NETAPP INC',
   'cusip6': '64110D'}}]

In [60]:
kg.query("""
  MATCH (sc:Address)
    WHERE sc.city = "Santa Clara"
  MATCH (com:Company)-[:LOCATED_AT]->(comAddr:Address)
    WHERE point.distance(sc.location, comAddr.location) < 10000
  RETURN com.companyName, com.companyAddress
""")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: companyAddress)} {position: line: 6, column: 31, offset: 205} for query: '\n  MATCH (sc:Address)\n    WHERE sc.city = "Santa Clara"\n  MATCH (com:Company)-[:LOCATED_AT]->(comAddr:Address)\n    WHERE point.distance(sc.location, comAddr.location) < 10000\n  RETURN com.companyName, com.companyAddress\n'


[]

In [61]:
kg.query("""
  MATCH (address:Address)
    WHERE address.city = "Santa Clara"
  MATCH (mgr:Manager)-[:LOCATED_AT]->(managerAddress:Address)
    WHERE point.distance(address.location, 
        managerAddress.location) < 10000
  RETURN mgr.managerName, mgr.managerAddress
""")

[]

In [62]:
# Which investment firms are near Palo Aalto Networks?
kg.query("""
  CALL db.index.fulltext.queryNodes(
         "fullTextCompanyNames", 
         "Palo Aalto Networks"
         ) YIELD node, score
  WITH node as com
  MATCH (com)-[:LOCATED_AT]->(comAddress:Address),
    (mgr:Manager)-[:LOCATED_AT]->(mgrAddress:Address)
    WHERE point.distance(comAddress.location, 
        mgrAddress.location) < 10000
  RETURN mgr, 
    toInteger(point.distance(comAddress.location, 
        mgrAddress.location) / 1000) as distanceKm
    ORDER BY distanceKm ASC
    LIMIT 10
""")

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.fulltext.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such fulltext schema index: fullTextCompanyNames}